# Connection to the OMDb API

Register on OMDb and get an API key
http://www.omdbapi.com/

In [ ]:
#add the API key
omdbapi = ""

# Configuration of API
serviceurl = "http://www.omdbapi.com/?"
apikey = '&apikey='+omdbapi

# Creating Database in MySQL

In [ ]:
# Creating database
import pymysql

connection = pymysql.connect(host = "localhost", user = "root", passwd = "")
cursor = connection.cursor()

create_database = '''CREATE DATABASE IF NOT EXISTS RottenPotatoesMovieDatabase'''
cursor.execute(create_database)
print('RottenPotatoesMovieDatabase database created!')

connection.commit()
connection.close()

In [ ]:
# Creating relations
import pymysql

# Connect to database
connection = pymysql.connect(host = "localhost", user = "root", passwd = "", database = "RottenPotatoesMovieDatabase")
cursor = connection.cursor()

create_movies = '''CREATE TABLE IF NOT EXISTS MOVIES 
                        (title VARCHAR(100), 
                        year INTEGER,
                        rated VARCHAR(20),
                        released DATE,
                        runtime INTEGER, 
                        genre VARCHAR(50),
                        director VARCHAR(50),
                        writer VARCHAR(50),
                        plot TEXT,
                        language VARCHAR(100),
                        awards VARCHAR(250),
                        poster MEDIUMBLOB,
                        imdbID CHAR(9),
                        country VARCHAR(30), 
                        metascore REAL, 
                        IMDBrating REAL, 
                        primary key(imdbID))'''
cursor.execute(create_movies)
print('MOVIES relation created!')

create_tvseries = '''CREATE TABLE IF NOT EXISTS TVSERIES 
                        (title VARCHAR(100), 
                        year INTEGER,
                        rated VARCHAR(20),
                        released DATE, 
                        genre VARCHAR(50),
                        director VARCHAR(50),
                        writer VARCHAR(50),
                        plot TEXT,
                        language VARCHAR(100),
                        country VARCHAR(30), 
                        awards VARCHAR(250),
                        poster MEDIUMBLOB, 
                        IMDBrating REAL,
                        imdbID CHAR(9),
                        seasons INTEGER,
                        primary key(imdbID))'''

cursor.execute(create_tvseries)
print('TVSERIES relation created!')

create_actors = '''CREATE TABLE IF NOT EXISTS ACTOR
                        (imdbID CHAR(7),
                        name VARCHAR(100),
                        bio TEXT,
                        quotes TEXT,
                        trademark TEXT,
                        birthdate DATE,
                        birthNotes VARCHAR(500),
                        headshot MEDIUMBLOB,
                        PRIMARY KEY(imdbID))'''

cursor.execute(create_actors)
print('ACTOR relation created!')

create_moviescast = '''CREATE TABLE IF NOT EXISTS MOVIESCAST
                        (movieimdbID CHAR(9),
                        castimdbID CHAR(7),
                        role VARCHAR(500),
                        FOREIGN KEY(movieimdbID) REFERENCES MOVIES(imdbID),
                        FOREIGN KEY(castimdbID) REFERENCES ACTOR(imdbID))'''

cursor.execute(create_moviescast)
print('MOVIESCAST relation created!')

create_tvseriescast = '''CREATE TABLE IF NOT EXISTS TVSERIESCAST
                        (tvseriesimdbID CHAR(9),
                        castimdbID CHAR(7),
                        role VARCHAR(500),
                        FOREIGN KEY(tvseriesimdbID) REFERENCES TVSERIES(imdbID),
                        FOREIGN KEY(castimdbID) REFERENCES ACTOR(imdbID))'''

cursor.execute(create_tvseriescast)
print('TVSERIESCAST relation created!')

create_users = '''CREATE TABLE IF NOT EXISTS USER
                        (userID INT AUTO_INCREMENT,
                        email VARCHAR(100) NOT NULL,
                        password VARCHAR(50) NOT NULL,
                        name VARCHAR(100),
                        username VARCHAR(20),
                        birthdate DATE,
                        avatar INT,
                        PRIMARY KEY(userID))'''

cursor.execute(create_users)
print('USER relation created!')

create_moviereviews = '''CREATE TABLE IF NOT EXISTS MOVIESREVIEW
                        (title VARCHAR(250) NOT NULL,
                        content TEXT,
                        rating REAL,
                        userID INT,
                        movieimdbID CHAR(9),
                        CHECK(rating > 0 AND rating <= 10),
                        FOREIGN KEY(userID) REFERENCES USER(userID),
                        FOREIGN KEY(movieimdbID) REFERENCES MOVIES(imdbID))'''

cursor.execute(create_moviereviews)
print('MOVIESREVIEW relation created!')

create_tvseriesreviews = '''CREATE TABLE IF NOT EXISTS TVSERIESREVIEW
                        (title VARCHAR(250) NOT NULL,
                        content TEXT,
                        rating REAL,
                        userID INT,
                        tvseriesimdbID CHAR(9),
                        CHECK(rating > 0 AND rating <= 10),
                        FOREIGN KEY(userID) REFERENCES USER(userID),
                        FOREIGN KEY(tvseriesimdbID) REFERENCES TVSERIES(imdbID))'''

cursor.execute(create_tvseriesreviews)
print('TVSERIESREVIEW relation created!')

connection.commit()
connection.close()

# Retrieving top 250 movies' title from IMDb

In [ ]:
#Retrieve top 250 movies

from imdb import IMDb, IMDbError
try:
    ia = IMDb()
    m = ia.get_top250_movies()   
    movies_list = []
    for mov in m:
        movies_list.append(str(mov))
except IMDbError as e:
    print(e)
    
print('Movie names collected: ', len(movies_list))

# Adding Movies to the Database

In [ ]:
# populating movies
def arrange_movie_data(title):
    import urllib.request, urllib.parse, urllib.error
    import json
    from dateutil.parser import parse
    import imdb
    from imdb import helpers 
    import pymysql
       
    #get movie data from OMDb
    movie = {}
    try:
        url = serviceurl + urllib.parse.urlencode({'t':title}) + apikey
        retrieved = urllib.request.urlopen(url)
        data = retrieved.read()
        json_data = json.loads(data)
        #if movie data received
        if json_data['Response'] == 'True':
            # check if present in database
            connection = pymysql.connect(host = "localhost", user = "root", passwd = "", database = "RottenPotatoesMovieDatabase")
            cursor = connection.cursor()
            get_movies = '''SELECT title FROM MOVIES WHERE imdbID LIKE  '%s' '''
            cursor.execute(get_movies %json_data['imdbID'])
            row = cursor.fetchone()
            print('IN DATABASE: ', row)
            if row is not None: 
                return 0, 0, 0
            connection.close()
                        
            actors_list = [x.strip() for x in str.split(json_data['Actors'], sep = ",")]
            #selecting required fields
            movie['title'] = json_data['Title']
            movie['year'] = json_data['Year']
            movie['rated'] = json_data['Rated']
            released = parse(json_data['Released'])
            movie['released'] = released.strftime("%Y-%m-%d")
            movie['runtime'] = int(json_data['Runtime'].split()[0])
            movie['genre'] = json_data['Genre']
            movie['director'] = json_data['Director']
            movie['writer'] = json_data['Writer']
            movie['plot'] = json_data['Plot']
            movie['language'] = json_data['Language']
            movie['awards'] = json_data['Awards']
            poster_url = json_data['Poster']
            img = urllib.request.urlopen(poster_url).read()
            movie['poster'] = (img, )
            movie['imdbID'] = json_data['imdbID']
            movie['country'] = json_data['Country']
            movie['metascore'] = json_data['Metascore']
            movie['IMDBrating'] = json_data['imdbRating']   
            print(title, ' data retrieved')
        else:
            print("Error encountered: ", json_data['Error'])
            return 0,0,0
    except urllib.error.URLError as e:
        print(f'ERROR: {e.reason}')
        
    #get actor's data
    actors = {}
    for actor in actors_list:
        uno = {}
        try:
            ia = imdb.IMDb()
            person = ia.search_person(actor)
            try:
                my_id = person[0].personID
            except IndexError as error:
                print('Actor not Found')
                continue
            p = ia.get_person(my_id)
        except imdb.IMDbError as e:
            print(e)
        uno['imdbID'] = p['imdbID']
        uno['name'] = p['name']
        try:
            uno['bio'] = p['mini biography']
        except KeyError:
            uno['bio'] = ''
        try:
            uno['quotes'] = str(p['quotes'])
        except KeyError:
            uno['quotes'] = ''
        try:
            uno['trademark'] = str(p['trade mark'])
        except KeyError:
            uno['trademark'] = ''
        try:
            uno['birthdate'] = p['birth date']
        except KeyError:
            uno['bithdate'] = ''
        try:
            uno['birthNotes'] = p['birth notes']
        except KeyError:
            uno['birthNotes'] = ''
        try:
            poster_url = p['headshot']
            img = urllib.request.urlopen(poster_url).read()
            uno['headshot'] = (img, )
        except KeyError:
            uno['headshot'] = ''
                       
        actors[actor] = uno
        print(actor, ' data retrieved!')
    
    #get roles
    roles = []
    i = imdb.IMDb()
    toString = imdb.helpers.makeObject2Txt(personTxt=u'%(currentRole)s')
    m = i.get_movie(movie['imdbID'][2:])
    for cast in m['cast']:
        if str(cast) in actors_list:
            role = toString(cast)
            roles.append((movie['imdbID'], actors[str(cast)]['imdbID'], role))
    print(len(roles), ' roles retrieved!')
       
    #return dict, dict, tuple
    return movie, actors, roles
            

In [ ]:
# add movie data to database
def populate_movie_data(movie, actors, roles):
    import pymysql
    connection = pymysql.connect(host = "localhost", user = "root", passwd = "", database = "RottenPotatoesMovieDatabase")
    cursor = connection.cursor()
    
    movie['title'] = connection.escape_string(movie['title'])
    get_movies = "SELECT title FROM MOVIES WHERE imdbID = '%s'"
    cursor.execute(get_movies %movie['imdbID'])
    row = cursor.fetchone()
    if row is None:
        insert_movie = '''INSERT INTO MOVIES (title, year, rated, released, runtime, 
        genre, director, writer, plot, language, awards, poster, imdbID, country, metascore, IMDBrating)
                            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
        cursor.execute(insert_movie, tuple(movie.values()))
        print(movie['title'], ' added!')
    insert_actor = '''INSERT INTO ACTOR (imdbID, name, bio, quotes, trademark, birthdate, birthNotes, headshot)
                                VALUES(%s, %s, %s, %s, %s, %s, %s, %s)'''
    # fetches tuple with given imdbID
    get_actor_id = '''SELECT imdbID FROM ACTOR WHERE imdbID = '%s'  '''
    for actor in actors.keys():
        cursor.execute(get_actor_id %actors[actor]['imdbID'])
        row = cursor.fetchone()
        if row is None:
            cursor.execute(insert_actor, tuple(actors[actor].values()))
            print(actor, ' added!')
        else:
            print(actor, ' already added!')
          
    insert_movierole = '''INSERT INTO MOVIESCAST (movieimdbID, castimdbID, role) 
                                VALUES(%s, %s, %s)'''    
    for role in roles:
        cursor.execute(insert_movierole, role)
    print(len(roles), ' roles added . . . ')    
    connection.commit()
    connection.close()

## Add all the top 250 movies 

In [ ]:
import time, requests, socket
from pymysql import OperationalError
count = 0
for title in movies_list[126:]:
    try:  
        print('\nCOUNT: ', count)
        movie, actors, roles = arrange_movie_data(title)
        if movie == 0:
            count += 1
            continue
        print('\nADDING TO DATABASE NOW . . .')
        populate_movie_data(movie, actors, roles)
        count += 1
    except requests.Timeout as err:
        print("******* WAITING *******")
        time.sleep(60)
        
    except requests.RequestException as err:
        print("******* WAITING *******")
        time.sleep(60)
    except OperationalError:
        print('Skipped!')
    except socket.timeout:
        print('timeout . . . skipped!')
    except KeyError:
        print('Skipped!')

## Add movie by title

In [ ]:
title = "Scream"
movie, actors, roles = arrange_movie_data(title)
if movie != 0:
    populate_movie_data(movie, actors, roles)
else:
    print('movie already added!')

## List of TV series to add

In [ ]:
series = ["Monty Python's Flying Circus", "All in the Family", "The Jeffersons", "Lonesome Dove", 
          "The Simpsons", "Seinfeld", "Friends", "The Sopranos", "Curb Your Enthusiasm", "Arrested Development", 
          "House Hunters", "Lost", "Avatar: The Last Airbender", "Doctor Who", "Mad Men", "The Big Bang Theory", 
          "Breaking Bad", "American Horror Story", "BoJack Horseman", "Riverdale", "Narcos: Mexico"]

# Add TV series to the database

In [ ]:
# populating tvseries
def arrange_tvseries_data(title):
    import urllib.request, urllib.parse, urllib.error
    import json
    from dateutil.parser import parse
    import imdb
    from imdb import helpers 
    import pymysql
    
    #get tvseries data from OMDb
    tvseries = {}
    try:
        url = serviceurl + urllib.parse.urlencode({'t':title}) + apikey
        retrieved = urllib.request.urlopen(url)
        data = retrieved.read()
        json_data = json.loads(data)
        #if tvseries data received
        if json_data['Response'] == 'True':
            # check if present in database
            connection = pymysql.connect(host = "localhost", user = "root", passwd = "", database = "RottenPotatoesMovieDatabase")
            cursor = connection.cursor()
            get_movies = '''SELECT title FROM TVSERIES WHERE imdbID LIKE  '%s' '''
            cursor.execute(get_movies %json_data['imdbID'])
            row = cursor.fetchone()
            print('IN DATABASE: ', row)
            if row is not None: 
                return 0, 0, 0
            connection.close()
                       
            actors_list = [x.strip() for x in str.split(json_data['Actors'], sep = ",")]
            #selecting required fields
            tvseries['title'] = json_data['Title']
            tvseries['year'] = json_data['Year']
            tvseries['rated'] = json_data['Rated']
            if json_data['Released'] != 'N/A':
                released = parse(json_data['Released'])
                tvseries['released'] = released.strftime("%Y-%m-%d")
            tvseries['genre'] = json_data['Genre']
            tvseries['director'] = json_data['Director']
            tvseries['writer'] = json_data['Writer']
            tvseries['plot'] = json_data['Plot']
            tvseries['language'] = json_data['Language']
            tvseries['country'] = json_data['Country']
            tvseries['awards'] = json_data['Awards']
            poster_url = json_data['Poster']
            img = urllib.request.urlopen(poster_url).read()
            tvseries['poster'] = (img, )
            tvseries['IMDBrating'] = json_data['imdbRating'] 
            tvseries['imdbID'] = json_data['imdbID']
            tvseries['seasons'] = json_data['totalSeasons']
            print(title, ' data retrieved!')
        else:
            print("Error encountered: ", json_data['Error'])           
    except urllib.error.URLError as e:
        print(f'ERROR: {e.reason}')
    
    #get roles
    roles = []
    actors_ids = []
    i = imdb.IMDb()
    toString = imdb.helpers.makeObject2Txt(personTxt=u'%(currentRole)s')
    m = i.get_movie(tvseries['imdbID'][2:])
    
    for cast in m['cast']:
        actor = str(cast)
        ia = imdb.IMDb()
        person = ia.search_person(actor)
        try:
            my_id = person[0].personID
            actors_ids.append(my_id)
        except IndexError as error:
            print('Actor not Found')
            continue
        role = toString(cast)
        roles.append((tvseries['imdbID'], my_id, role))
    print(len(roles), ' roles retrieved!')
    
    #get actor's data
    actors = {}
    for actor_id in actors_ids:
        uno = {}
        try:
            ia = imdb.IMDb()
            p = ia.get_person(actor_id)
        except imdb.IMDbError as e:
            print(e)
        uno['imdbID'] = p['imdbID']
        uno['name'] = p['name']
        try:
            uno['bio'] = str(p['mini biography'])
        except KeyError:
            uno['bio'] = ''
        try:
            uno['quotes'] = str(p['quotes'])
        except KeyError:
            uno['quotes'] = ''
        try:
            uno['trademark'] = str(p['trade mark'])
        except KeyError:
            uno['trademark'] = ''
        try:
            uno['birthdate'] = str(p['birth date'])
        except KeyError:
            uno['bithdate'] = ''
        try:
            uno['birthNotes'] = str(p['birth notes'])
        except KeyError:
            uno['birthNotes'] = ''
        try:
            poster_url = p['headshot']
            img = urllib.request.urlopen(poster_url).read()
            uno['headshot'] = (img, )
        except KeyError:
            uno['headshot'] = ''                
        if len(uno.keys()) == 8:
            actors[uno['name']] = uno 
            print(uno['name'], ' data retrieved!')
        else:
            print(uno['name'], 'skipped!')
            
    #return dict, dict, tuple
    
    return tvseries, actors, roles
            

In [ ]:
# add tvseries data to database
def populate_tvseries_data(tvseries, actors, roles):
    import pymysql
    connection = pymysql.connect(host = "localhost", user = "root", passwd = "", database = "RottenPotatoesMovieDatabase")
    cursor = connection.cursor()
    #cursor.execute('SET innodb_lock_wait_timeout = 1000000')
    #cursor.execute('SET GLOBAL max_allowed_packet=67108864')
    
    tvseries['title'] = connection.escape_string(tvseries['title'])
    get_tvseries = "SELECT title FROM TVSERIES WHERE imdbID = '%s'"
    cursor.execute(get_tvseries %tvseries['imdbID'])
    row = cursor.fetchone()
    if row is None:
        insert_tvseries = '''INSERT INTO TVSERIES (title, year, rated, released, genre, director, 
        writer, plot, language, country, awards, poster, IMDBrating, imdbID, seasons)
                            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
        cursor.execute(insert_tvseries, tuple(tvseries.values()))
        print(tvseries['title'], ' added!')
    insert_actor = '''INSERT INTO ACTOR (imdbID, name, bio, quotes, trademark, birthdate, birthNotes, headshot)
                                VALUES(%s, %s, %s, %s, %s, %s, %s, %s)'''
    # fetches tuple with given imdbID
    get_actor_id = '''SELECT imdbID FROM ACTOR WHERE imdbID = '%s'  '''
    for actor in actors.keys():
        cursor.execute(get_actor_id %actors[actor]['imdbID'])
        row = cursor.fetchone()
        if row is None:
            cursor.execute(insert_actor, tuple(actors[actor].values()))
            print(actor, ' added!')
        else:
            print(actor, ' already added!')
          
    insert_tvseriesrole = '''INSERT INTO TVSERIESCAST (tvseriesimdbID, castimdbID, role) 
                                VALUES(%s, %s, %s)'''    
    for role in roles:
        cursor.execute(insert_tvseriesrole, role)
    print(len(roles), ' roles added . . . ')    
    connection.commit()
    connection.close()

## Add the TV series from list

In [ ]:
#Will contain the series that fetch error or are already added to the database
skipseries = []

In [ ]:
import time, requests, socket
from pymysql import OperationalError
count = 0
for title in series:
    if title not in skipseries:
        try:  
            print('\nCOUNT: ', count)
            tvseries, actors, roles = arrange_tvseries_data(title)
            if tvseries == 0:
                count += 1
                skipseries.append(title)
                continue
            print('\nADDING TO DATABASE NOW . . .')
            populate_tvseries_data(tvseries, actors, roles)
            count += 1
            skipseries.append(title)
        except requests.Timeout as err:
            print("******* WAITING *******")
            time.sleep(60)
            skipseries.append(title)
        except requests.RequestException as err:
            print("******* WAITING *******")
            time.sleep(60)
            skipseries.append(title)
        except OperationalError:
            print('Skipped!')
            skipseries.append(title)
        except socket.timeout:
            print('timeout . . . skipped!')
            skipseries.append(title)
        except KeyError:
            print('Skipped!')
            skipseries.append(title)
        except TypeError:
            print('Skipped!')
            skipseries.append(title)
    else:
        print(title)
        print("***** SKIPPED ******")

## Add TV Series by Title

In [ ]:
title = "Two and a half  men"
tvseries, actors, roles = arrange_tvseries_data(title)
if tvseries != 0:
    populate_tvseries_data(tvseries, actors, roles)
else:
    print('tvseries already added!')